This notebook implements the 6-step / 8-step FFT (fft) algorithm as provided in [OTFFT](http://wwwa.pikara.ne.jp/okojisan/otfft-en/stockham2.html). Accordingly, the inverse FFT (ifft) algorithm will be implemented as well. When the input of FFT, `T` with length `n`, consists of real-valued elements only, we can take advantage of real FFT (rfft) as explained in [2.6.2](https://www.researchgate.net/profile/Christos-Bechlioulis/publication/341270520_FFT_algorithms_are_not_mine_However_I_am_going_to_convince_you_soon_regarding_the_visit_of_RMS_to_our_university_Believe_it_or_not_this_is_me_This_is_us_Univeristy_of_Patras_you_have_chosen_a_quite_wr/links/5fa53ce7299bf10f7328c33b/FFT-algorithms-are-not-mine-However-I-am-going-to-convince-you-soon-regarding-the-visit-of-RMS-to-our-university-Believe-it-or-not-this-is-me-This-is-us-Univeristy-of-Patras-you-have-chosen-a-quite.pdf). 

In [2]:
import math
import time

import numba
import numpy as np
import matplotlib.pyplot as plt
import scipy

from numba import njit, prange
import numpy.testing as npt

from stumpy import core

Let's start with `rfft`. First, we write the test!

## rfft

In [3]:
def test_rfft(n_powers_list):
    seed = 0
    np.random.seed(seed)
    for p in n_powers_list:
        n = 2 ** p
        T = np.random.rand(n)
        
        ref = scipy.fft.rfft(T)
        comp = rfft(T)
        
        npt.assert_almost_equal(ref, comp)

We now implement `rfft` function according to the steps provided in [2.6.2](https://www.researchgate.net/profile/Christos-Bechlioulis/publication/341270520_FFT_algorithms_are_not_mine_However_I_am_going_to_convince_you_soon_regarding_the_visit_of_RMS_to_our_university_Believe_it_or_not_this_is_me_This_is_us_Univeristy_of_Patras_you_have_chosen_a_quite_wr/links/5fa53ce7299bf10f7328c33b/FFT-algorithms-are-not-mine-However-I-am-going-to-convince-you-soon-regarding-the-visit-of-RMS-to-our-university-Believe-it-or-not-this-is-me-This-is-us-Univeristy-of-Patras-you-have-chosen-a-quite.pdf).

In [4]:
def _rfft(T):
    """
    For the input `T` with length `n=len(T)`, this function returns its
    real fast fourier transform (rfft) with length of `(n // 2) + 1`.
    
    Parameters
    ----------
    T : numpy.ndarray
        A time series of interest, with real-valued numbers
    
    Returns
    -------
    out : numpy.ndarray
        the real fast fourier transform (rfft) of input `T`
    """
    n = len(T)
    half_n = int(n // 2)
    
    x = T[::2] + 1j * T[1::2]
    x[:] = scipy.fft.fft(x)  # we will implement our fft shortly!
    
    out = np.empty(half_n + 1, dtype=np.complex_)
    out[0] = x[0].real + x[0].imag
    out[half_n] = x[0].real - x[0].imag
    out[n // 4] = x[n // 4].conjugate()
    
    theta0 = 2 * math.pi / n
    for k in range(1, n // 4):
        theta = theta0 * k
        a =  x[half_n - k].conjugate()
        b = 0.5 * (x[k] - a) * (1.0 + complex(math.sin(theta), math.cos(theta)))
        out[k] = x[k] - b
        out[half_n - k] = (a + b).conjugate()
    
    return out

In [5]:
def rfft(T):
    """
    For the input `T` with length `n=len(T)`, this function returns its
    real fast fourier transform (rfft) with length of `(n // 2) + 1`.
    
    Parameters
    ----------
    T : numpy.ndarray
        A time series of interest, with real-valued numbers
    
    Returns
    -------
    out : numpy.ndarray
        the real fast fourier transform (rfft) of input `T`
    """
    return _rfft(T)

In [6]:
n_powers_list = np.arange(2, 11)
test_rfft(n_powers_list)

We now work on implementing 6-step / 8-step FFT algorithm. We then revisit the two functions above. We will replace `scipy.fft.fft(x)` with our FFT function, and then change them accordingly.

In [133]:
@njit(fastmath=True)
def _fft0(n, s, eo, x, y):
    """
    A recurive function that is being called by six-step / eight-step FFT algorithm, 
    and update `x` in place.
    
    Parameters
    ----------
    n : int
        Length of sequence
    
    s : int
        size of striding window
    
    eo : bool
        If False, `x` is output. If True, `y` is the output.
        
    x : numpy.ndarray
        A 1D numpy.ndarray with `np.complex_` dtype
        
    y : nummpy.ndarray
        A 1D numpy.ndarray with same size as `x` and `np.complex_` dtype. 
    
    Returns
    -------
    
    Notes
    -----
    <http://wwwa.pikara.ne.jp/okojisan/otfft-en/sixstepfft.html>`__

    See function `fft0` provided in "List-11: Six-Step FFT"
    """
    if n == 2:
        if eo:
            z = y
        else:
            z = x
        
        for i in range(s):
            j = i + s
            a = x[i]
            b = x[j]
            z[i] = a + b
            z[j] = a - b
            
    elif n >= 4:
        m = n // 2
        sm = s * m
        
        theta = 2 * math.pi / n
        c = complex(math.cos(theta), -math.sin(theta))
        
        twiddle_factor = 1.0
        for p in range(m):
            sp = s * p
            two_sp = 2 * sp
            for q in range(s):
                i = sp + q
                j = i + sm
                
                k = two_sp + q
                y[k] = x[i] + x[j]
                y[k + s] = (x[i] - x[j]) * twiddle_factor
        
            twiddle_factor = twiddle_factor * c
        
        _fft0(m, 2 * s, not eo, y, x)
        
    else:
        pass